[Data Leakage](https://www.kaggle.com/code/alexisbcook/data-leakage)
>Find and fix this probelm that ruins your model in subtle ways.  
 교묘한 방법으로 모델을 망치는 문제를 찾아 해결하자.

# Introuduction
이번에는 **데이터 유출(Data Leakage)**과 이를 방지하는 방법을 배울 것이다.  
만약 예방법을 모른다면 유출이 자주 발생하며, 교묘하고 위험한 방식으로 모델을 망치게 된다.  
따라서 이는 data science를 연습하는데 가장 중요한 개념중 한 가지이다. 

**Data Leakage(데이터 유출,누수)란?**  <br>
**Data Leakage**(or**leakage**, 이하 누출)은 훈련 데이터에 대상에 대한 정보가 포함되어 있을때 발생한다.(모델을 예측에 사용할때는 유사한 데이터를 사용할 수 없다.)  
이는 훈련 세트(검증 데이터 포함)에서 높은 성능을 발휘하지만, 모델은 프로덕션에서는 형편없는 성능을 보일 것 이다.   

다시 말해, 누출을 사용하면 모형을 사용하면 결정을 내리기 전까지 정확해 보이지만 결국 모델은 매우 부정확해진다. 

누출에는 두 가지 종류가 있는데, 이는 **target leakage**와 **train-test-contamination**.이다. 

## Target Leakage
**Target Leakage**은 예측 변수에 만들 당시에 사용할 수 없는 데이터가 포함될 때 발생한다.  
특징(feature)이 좋은 예측을 만드는데 도움이 되는지 여부가 아니라, 데이터를 사용할 수 있게 되는 타이밍 또는 시간 순서의 관점에서 목표 누출에 대해 생각하는 것이 중요하다.   

한 가지 예시가 도움이 될 것이다. 누가 폐렴에 걸릴지 예측하고 싶다고 상상해보자.  
원 데이터의 상위 몇 행은 다음과 같다.  


|got_pneumonia	|age|	weight|	male	|took_antibiotic_medicine|	...|
|--|--|--|--|--|--|
|False|	65|	100|	False|	False|	...|
|False|	72|	130|	True|	False|	...|
|True|	58|	100|	False|	True|...|

사람들은 건강 회복을 위해 폐렴에 걸린 후 상생제를 복용한다. 원 데이터는 이러한 열 간의 강한 관계를 보여주지만  
**took_antibiotic_medicine**은 **got_pneumonia**값이 결정된 후 자주 변경된다. 이것이 **target leakage(타겟 누출)**이다.  

모델은 take_antibiotic_medicine(항생제)에 대한 값이 False인 사람은 폐렴에 걸리지 않았음을 알 수 있다.  
검증 데이터는 훈련 데이터와 동일한 소스에서 오기 때문에 패턴은 검증에서 자체적으로 반복되고 모델은 우수한 검증(또는 교차 검증) 점수를 갖게 된다. 


하지만 이후에 현실 세계에 적용된다면 매우 부정확할 것이다. 왜냐하면 폐렴에 걸릴 환자라도 미래의 건강에 대해 예측해야 할 때 아직 항생제를 투여받지 않았기 때문이다.  
이러한 데이터 유출을 방지하기 위해서는 목표값이 실현된 업데이트(또는 생성)된 변수를 제외해야 한다.  
![nn](https://i.imgur.com/y7hfTYe.png![image.png](attachment:image.png))

## Train-Test Contamination
**train-Test 오염**

훈련 데이터와 검증 데이터를 구분하지 않으면 다른 유형의 누출이 발생한다.  

검증은 모델이 이전에 고려하지 않은 데이터에 대해 수행하는 방식을 측정하기 위한 것임을 기억해야 한다.  
검증 데이터가 전처리 동작에 영향을 미치는 경우 이 프로세스를 미묘한 방식으로 손상시킬 수 있다.  
이는 **train-test contamination**이라 한다.  

예를 들어, **train_test_split()**을 호출하기 전에 전처리(결측값 대치 등)를 실행한다고 가정해보자.  
최종 결과는? 모델은 높은 검증 점수를 받아 신뢰도를 높일수는 있지만, 결정을 내리기 위해 모델을 배포할 때는 성능이 저하된다.  

결국, 검증 혹은 테스트 데이터를 예측 방법에 통합했으므로 새 데이터로 일반화할 수 없는 경우에도 특정 데이터에 대해 더 잘 예측할 수 있다.  
이 문제는 더 복잡한 feature engineering 수행시 훨씬 더 교묘하고 위험해진다.  

검증이 단순한 train-test split을 기반으로 하는 경우 전처리 단계의 fitting을 포함해 모든 유형의 fitting에서 검증 데이터를 제외해야 한다.  
이는 scikit-learn 파이프 라인을 사용하면 더 수월하다.  교차 검증을 수행할 때는 파이프라인 내에서 전처리를 수행하는 것이 훨씬 중요하다.  

# Example
예시에서는 target leakage를 탐지하고 제거하는 한 가지 방법을 배우게 된다.  

우리는 신용카드 애플리케이션에 대한 데이터 세트를 사용하고 *기본 데이터 설정 코드를 생략할 것이다. *  
최종 결과는 각 신용카드 애플리케이션에 대한 정보가 데이터 프레임 X에 저장된다는 것이다.  
이를 사용해 시리즈


*신용카드 응용 프로그램*에 대한 데이터 세트를 사용하고 기본 데이터 설정 코드는 건너뛴다.  
최종 결과는 각 신용 카드 응용 프로그램에 대한 정보가 DataFrame X에 저장된다는 것이다.  
이를 사용해 Series y에서 승인된 응용 프로그램을 예측한다. 

## 데이터 누출 있음

In [73]:
import pandas as pd

# Read the data
data=pd.read_csv('./data/credit_card.csv')

# Select target
y=data.card

# Select predictors
X=data.drop(['card'], axis=1)

print(f"Number of rows in the dataset : {X.shape[0]}")

Number of rows in the dataset : 1319


In [2]:
data.head()

,card,reports,age,income,share,expenditure,owner,selfemp,dependents,months,majorcards,active
0,yes,0,37.66667,4.5200,0.033270,124.983300,yes,no,3,54,1,12
1,yes,0,33.25000,2.4200,0.005217,9.854167,no,no,3,34,1,13
2,yes,0,33.66667,4.5000,0.004156,15.000000,yes,no,4,58,1,5
3,yes,0,30.50000,2.5400,0.065214,137.869200,no,no,0,25,1,7
4,yes,0,32.16667,9.7867,0.067051,546.503300,yes,no,2,64,1,5


위 데이터는 작은 데이터 세트이기 때문에 모델의 정확한 측정을 보장하기 위해 교차 검증을 사용할 것이다. 

In [59]:
# 범주형 데이터 
cat_cols=[cname for cname in X.columns if X[cname].nunique() < 10 and cname!='card']


## 선택된 열
cols = cat_cols + num_cols
X=[cols].copy()

In [77]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# 수치형 데이터
num_cols=[cname for cname in X.columns if X[cname].nunique() >=10]
##수치형 데이터 전처리
num_transform=StandardScaler()

# 범주형 데이터 
cat_cols=[cname for cname in X.columns if X[cname].nunique() < 10 and cname!='card']
##범주형 데이터 전처리
cat_transform=OneHotEncoder(handle_unknown='ignore')

## 수치/범주형 데이터 전처리 번들
preprocessor=ColumnTransformer(
  transformers=[
      ('num',num_transform, num_cols),
      ('cat',cat_transform,cat_cols)
  ])

# 기본 모델
model = RandomForestClassifier(n_estimators=100)

In [80]:
# 교차검증
pipes=make_pipeline(preprocessor, model)
cv_scores=cross_val_score(pipes, X, y, cv=5, scoring='accuracy')
print(f"Cross-validation accuracy : {cv_scores.mean()}")

Cross-validation accuracy : 0.9818095402696164


경험상 98%의 정확도를 가진 모델을 찾는 일은 매우 드물며,target leakage 되지 않도록 데이터를 면밀히 조사해야 할 만큼 흔치 않은 일이다.  

다음은 데이터 특징 요약이다.  
* card: 신용카드 신청이 받아들여지면 1, 받아들여지지 않으면 0
* report: 주요비하보고서수
* age: 나이와 12분의 1을 더한 나이
* income : 연간소득(나눗셈 10,000)
* share: 연소득 대비 월별 신용카드 지출액 비율
* expenditure: 월평균신용카드지출
* owners: 집을 소유한 경우 1개, 임대인 경우0
* selfempl: 자영업자일 경우 1, 그렇지 않을 경우 0
* dependents: 요소: 1 + 부양자 수
* months: 현재 주소에 거주하는 월
* majorvards: 보유 주요 신용 카드 수
* active: 활성 신용 계정 수


몇 가지 변수가 의심스러워 보인다. 예를 들어 **expenditure**은 이 카드에 대한 지출인지 혹은 이전 카드에 의한 지출인지 알 수 없다.  

이 시점에서 기본 데이터 비교는 매우 유용할 수 있다. 

In [96]:
expenditures_cardholders=X.expenditure.loc[y=='yes']
expenditures_noncardholders=X.expenditure.loc[y=='no']

print('Fraction of those who did not receive a card and had no expenditures: %.2f' \
      %((expenditures_noncardholders == 0).mean()))
print('Fraction of those who received a card and had no expenditures: %.2f' \
      %(( expenditures_cardholders == 0).mean()))

Fraction of those who did not receive a card and had no expenditures: 1.00
Fraction of those who received a card and had no expenditures: 0.02


In [ ]:
expenditure_cardholders=X.e

* 위처럼 카드를 받지 않은 사람은 모두 지출이 없었고, 카드를 받은 사람은 2%만 지출이 없었다.   

우리 모델이 높은 정확도를 보인다는 것은 놀라운 일이 아니다.  
그러나 이는 신청한 카드에 대한 지출을 의미하는 target leakage의 경우로 보인다.  

*share*는 *expenditure*에 의해 부분적으로 결정되기에 이것도 제외되어야 한다.  
*active* 및 *majorcard* 변수는 다소 명확하지 않지만 설명에서 보면 우려되는 것으로 보인다.  
대부분의 경우 데이터를 만든 사람을 추적해 더 많은 정보를 찾을 수 없다면 후회하는 것보단 안전한 것이 좋다.  


## 데이터 누출 없음.

In [103]:
potential_leaks = ['expenditure', 'share', 'active', 'majorcards']
X2 = X.drop(potential_leaks, axis=1)

# 수치형 데이터
num_cols2=[cname for cname in X2.columns if X2[cname].nunique() >=10]
##수치형 데이터 전처리
num_transform=StandardScaler()

# 범주형 데이터 
cat_cols2=[cname for cname in X2.columns if X2[cname].nunique() < 10 and cname!='card']
##범주형 데이터 전처리
cat_transform=OneHotEncoder(handle_unknown='ignore')

## 수치/범주형 데이터 전처리 번들
preprocessor=ColumnTransformer(
  transformers=[
      ('num',num_transform, num_cols2),
      ('cat',cat_transform,cat_cols2)
  ])

# 기본 모델
model = RandomForestClassifier(n_estimators=100)

In [107]:
pipes=make_pipeline(preprocessor, model)
cv_scores=cross_val_score(pipes, X2, y, cv=5, scoring='accuracy')
print(f"Cross-validation accuracy : \033[94m{cv_scores.mean()}")

Cross-validation accuracy : 0.827886277220878


이 정확도는 상당히 낮아 실망스러울 수도 있다.  
그러나 새 응용 프로그램에 사용될 때 80%의 정확도를 예상할 수 있는 반면, 누출 모델은 (교차 검증 점수가 높음에도) 그보다 훨씬 나쁜 결과를 나타낼 것이다. 


# Conclusion
데이터 유출은 많은 데이터 과학 응용 프로그램에서 수백만 달러 실수가 될 수 있다.  
훈련 및 검증 데이터를 신중하게 분리하면 train-test contamination을 방지할 수 있으며, 파이프라인은 이러한 분리를 구현하는데 도움이 될 수 있다.  
마찬가지로 주의, 상식 및 데이터 탐색의 조합은 대상 누출을 식별하는데 도움이 될 수 있다. 